In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

### 모델 학습

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 이미지 파일 경로
image_dir = '/content/drive/MyDrive/mnvision/Good'

In [ ]:
# 하이퍼파라미터 설정
BUFFER_SIZE = 60000
BATCH_SIZE = 32
EPOCHS = 2000
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16

In [ ]:
# 이미지 데이터셋 로딩 및 전처리
def load_images(image_dir):
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')]
    images = [tf.image.decode_jpeg(tf.io.read_file(file), channels=3) for file in image_files]
    images = [tf.image.resize(image, (480, 480)) for image in images]  # 이미지 크기 조정
    images = np.array(images)
    images = (images - 127.5) / 127.5  # [-1, 1]로 정규화
    return images

In [ ]:
# 생성기 정의
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(30*30*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((30, 30, 256)))
    assert model.output_shape == (None, 30, 30, 256)

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 60, 60, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 120, 120, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 240, 240, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 480, 480, 3)

    return model

In [ ]:
# 판별기 정의
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[480, 480, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
# 손실 함수 정의
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# 옵티마이저 정의
generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# 학습 데이터 준비
train_images = load_images(image_dir)
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# 학습 루프
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
# 모델 학습
for epoch in range(EPOCHS):
  # if epoch % 10 == 0:
  #   print(f'Epoch {epoch}/{EPOCHS}')
  print(f'Epoch {epoch}/{EPOCHS}')
  for image_batch in train_dataset:
      train_step(image_batch)

  if epoch % 100 == 0:
      noise = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])
      generated_images = generator(noise, training=False)
      fig = plt.figure(figsize=(4, 4))

      for i in range(generated_images.shape[0]):
          plt.subplot(4, 4, i+1)
          plt.imshow((generated_images[i, :, :, :].numpy() * 127.5 + 127.5).astype(np.uint8))
          plt.axis('off')

      plt.show()

### 이미지 생성 모델 불러오기

In [ ]:
# 생성기 모델 저장
generator.save('/content/drive/MyDrive/mnvision/generator_model')

In [ ]:
generator = tf.keras.models.load_model('/content/drive/MyDrive/mnvision/generator_model')

In [ ]:
def save_generated_images(epoch, generator, num_examples_to_generate, noise_dim, output_dir):
    noise = tf.random.normal([num_examples_to_generate, noise_dim])
    generated_images = generator(noise, training=False)
    for i in range(num_examples_to_generate):
        image = generated_images[i, :, :, :] * 127.5 + 127.5
        image = tf.cast(image, tf.uint8)
        file_path = os.path.join(output_dir, f'generated_image_epoch_{epoch}_sample_{i}.png')
        tf.keras.preprocessing.image.save_img(file_path, image)
        print(f'Saved {file_path}')

# 이미지를 저장할 디렉토리 지정
output_dir = '/content/drive/MyDrive/mnvision/generated_images'
os.makedirs(output_dir, exist_ok=True)

# 모델 학습 루프에서 호출하여 저장된 이미지를 생성하고 저장
for epoch in range(EPOCHS):
    for image_batch in train_dataset:
        train_step(image_batch)

    if epoch % 100 == 0:
        save_generated_images(epoch, generator, NUM_EXAMPLES_TO_GENERATE, NOISE_DIM, output_dir)


In [ ]:
def plot_generated_images(epoch, generator, num_examples_to_generate, noise_dim):
    noise = tf.random.normal([num_examples_to_generate, noise_dim])
    generated_images = generator(noise, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((generated_images[i, :, :, :] * 127.5 + 127.5).numpy().astype(np.uint8))
        plt.axis('off')
    plt.suptitle(f'Generated Images (Epoch {epoch})', fontsize=16)
    plt.show()

# 특정 에포크에서 생성된 이미지 출력
plot_generated_images(1000, generator, NUM_EXAMPLES_TO_GENERATE, NOISE_DIM)
